In [4]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
cols = ['text','sentiment']
df = pd.read_csv("C:/Users/dell/Desktop/imdb.csv",header=None, names=cols,encoding = 'unicode_escape')
# above line will be different depending on where you saved your data, and your file name
print(df.head())
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

df['sentiment'] = df['sentiment'].astype(np.int64)
#df.text[0]
df.info()
x = df.text
y = df.sentiment

from sklearn.cross_validation import train_test_split
SEED = 500
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.6, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)
print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))




from sklearn.feature_extraction.text import TfidfVectorizer
tvec1 = TfidfVectorizer(stop_words='english',ngram_range=(1, 3),max_features=10000)
x_train_tfidf = tvec1.fit_transform(x_train)
x_validation_tfidf = tvec1.transform(x_validation).toarray()
x1_train = {"great plot. everyone should watch"}
x1_train_tfidf = tvec1.transform(x1_train).toarray()
#tvec1.vocabulary
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import tensorflowjs as tfjs

def batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            counter=0

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=5654))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit_generator(generator=batch_generator(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

tfjs.converters.save_keras_model(model,"C:/Users/dell/Desktop/tens_senti_analysis")
print(model.summary())
model.predict(x1_train_tfidf)

                                                text  sentiment
0  A very, very, very slow-moving, aimless movie ...        0.0
1  Not sure who was more lost - the flat characte...        0.0
2  Attempting artiness with black & white and cle...        0.0
3       Very little music or anything to speak of.          0.0
4  The best scene in the movie was when Gerardo i...        1.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 2 columns):
text         999 non-null object
sentiment    999 non-null int64
dtypes: int64(1), object(1)
memory usage: 15.7+ KB
Train set has total 399 entries with 49.37% negative, 50.63% positive
Validation set has total 300 entries with 49.67% negative, 50.33% positive
Test set has total 300 entries with 51.33% negative, 48.67% positive
Epoch 1/5
13/12 [===============================] - 1s 73ms/step - loss: 0.6930 - acc: 0.4928 - val_loss: 0.6896 - val_acc: 0.5533
Epoch 2/5
13/12 [===============================] -

array([[0.44378355]], dtype=float32)

In [24]:
from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)
print(x_test)

[list([1, 591, 202, 14, 31, 6, 717, 10, 10, 18142, 10698, 5, 4, 360, 7, 4, 177, 5760, 394, 354, 4, 123, 9, 1035, 1035, 1035, 10, 10, 13, 92, 124, 89, 488, 7944, 100, 28, 1668, 14, 31, 23, 27, 7479, 29, 220, 468, 8, 124, 14, 286, 170, 8, 157, 46, 5, 27, 239, 16, 179, 15387, 38, 32, 25, 7944, 451, 202, 14, 6, 717])
 list([1, 14, 22, 3443, 6, 176, 7, 5063, 88, 12, 2679, 23, 1310, 5, 109, 943, 4, 114, 9, 55, 606, 5, 111, 7, 4, 139, 193, 273, 23, 4, 172, 270, 11, 7216, 10626, 4, 8463, 2801, 109, 1603, 21, 4, 22, 3861, 8, 6, 1193, 1330, 10, 10, 4, 105, 987, 35, 841, 16873, 19, 861, 1074, 5, 1987, 17975, 45, 55, 221, 15, 670, 5304, 526, 14, 1069, 4, 405, 5, 2438, 7, 27, 85, 108, 131, 4, 5045, 5304, 3884, 405, 9, 3523, 133, 5, 50, 13, 104, 51, 66, 166, 14, 22, 157, 9, 4, 530, 239, 34, 8463, 2801, 45, 407, 31, 7, 41, 3778, 105, 21, 59, 299, 12, 38, 950, 5, 4521, 15, 45, 629, 488, 2733, 127, 6, 52, 292, 17, 4, 6936, 185, 132, 1988, 5304, 1799, 488, 2693, 47, 6, 392, 173, 4, 21686, 4378, 270, 235